In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
from collections import deque
import datetime

import pyvisa

# --- 初始化TC290 ---
rm = pyvisa.ResourceManager()

TC290 = rm.open_resource("ASRL3::INSTR")
TC290.baud_rate = 115200
TC290.query('*IDN?')

def get_temperature():
    KRDG = TC290.query("KRDG? A")
    temperature = float(KRDG.strip())
    return temperature

# --- 初始化Dash应用 ---
app = dash.Dash(__name__)

# --- 数据存储 ---
# 使用deque可以高效地在列表两端添加或删除元素
# 这里我们设定图表最多只显示100个数据点
MAX_DATA_POINTS = 100
X_values = deque(maxlen=MAX_DATA_POINTS)
Y_values = deque(maxlen=MAX_DATA_POINTS)

# 初始数据
X_values.append(datetime.datetime.now())
Y_values.append(get_temperature())


# --- 应用布局 (Layout) ---
app.layout = html.Div([
    html.H1("Dash 实时监控图表"),
    
    # 图表组件，用于显示我们的实时数据
    dcc.Graph(id='live-update-graph'),
    
    # 核心组件：Interval计时器
    # 它会每1000毫秒（1秒）触发一次id='interval-component'的回调
    dcc.Interval(
        id='interval-component',
        interval=1 * 1000,  # in milliseconds
        n_intervals=0
    )
])


# --- 回调函数 (Callback) ---
# 这个回调函数将 dcc.Interval 的触发与 dcc.Graph 的更新连接起来
@app.callback(
    Output('live-update-graph', 'figure'),  # 输出：更新图表的figure属性
    [Input('interval-component', 'n_intervals')] # 输入：监听计时器的触发事件
)
def update_graph_live(n):
    # n_intervals 是一个从0开始计数的整数，代表计时器触发了多少次
    # 我们可以用它来决定是否更新数据，或者直接在函数里获取新数据
    
    current_time = datetime.datetime.now()
    new_y_value = get_temperature()
    
    # 更新我们的数据队列
    X_values.append(current_time)
    Y_values.append(new_y_value)
    
    # 创建图表对象
    fig = go.Figure(
        data=[go.Scatter(
            x=list(X_values),
            y=list(Y_values),
            mode='lines+markers',
            name='实时数据'
        )],
        layout=go.Layout(
            title='实时温度监控',
            xaxis=dict(title='时间'),
            yaxis=dict(title='温度 (K)', range=[299.7700, 299.8000]), # 固定Y轴范围
            showlegend=True
        )
    )
    
    return fig

# --- 启动应用 ---
if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
import pyvisa
rm = pyvisa.ResourceManager()
rm.list_resources()

In [1]:
def map_integer_to_trim_string(n):
    """
    将一个 0 到 4095 之间的整数映射为指定的 Trimming 字符串格式。

    Args:
        n (int): 要转换的整数。

    Returns:
        str: 格式化后的 Trimming 字符串。
        
    Raises:
        ValueError: 如果整数不在 0 到 4095 的范围内。
    """
    if not 0 <= n <= 4095:
        raise ValueError("输入整数必须在 0 到 4095 的范围内。")

    # 1. 将整数转换为 12 位的二进制字符串，不足 12 位则在左侧补 0
    #    bin_str[0]  是最高位 (b11 -> R1_IN3)
    #    bin_str[11] 是最低位 (b0  -> R3_IN0)
    bin_str = format(n, '012b')

    # 2. 根据二进制位和标签的对应关系，进行分组
    # R1 组的位: b11, b10, b9, b8 -> 对应 bin_str[0:4]
    # R2 组的位: b7, b6, b5, b4   -> 对应 bin_str[4:8]
    # R3 组的位: b3, b2, b1, b0   -> 对应 bin_str[8:12]
    r1_bits = bin_str[0:4]
    r2_bits = bin_str[4:8]
    r3_bits = bin_str[8:12]

    # 3. 输出格式要求颠倒每个组内的位的顺序
    # 例如 R1 组, 位顺序是 b11,b10,b9,b8, 输出顺序是 R1_IN0,R1_IN1,R1_IN2,R1_IN3
    # 这对应着 b8,b9,b10,b11, 恰好是 r1_bits 字符串的逆序

    # 反转每个组内的位顺序并直接拼接
    r1_o = ''.join(reversed(r1_bits))
    r2_o = ''.join(reversed(r2_bits))
    r3_o = ''.join(reversed(r3_bits))
    
    r1_output = '|'.join(reversed(r1_bits))
    r2_output = '|'.join(reversed(r2_bits))
    r3_output = '|'.join(reversed(r3_bits))

    # 4. 组合成最终的字符串
    return f"Trimming:{r1_output}|{r2_output}|{r3_output}|\r\n", f"{r1_o}{r2_o}{r3_o}"

def send_config(ser, config):

    """
    这个函数在一个已经打开的串口对象上发送命令并接收响应。
    """
    response_lines = []
    
    # 清空输入缓冲区，防止读取到上一次的残留数据
    ser.reset_input_buffer()
    
    # 发送命令
    command, Trimming = map_integer_to_trim_string(config)
    ser.write(command.encode("ascii"))
    # print(f"命令已发送: {command.strip()}")

    # 注意：这里的 sleep 可能仍然需要，取决于你的设备是否需要处理时间
    # 如果设备响应很快，可以尝试缩短或移除它
    # time.sleep(0.01) 

    # 循环读取，直到超时
    # while True:
    #     line = ser.readline()
    #     if line:
    #         response_lines.append(line.decode('ascii', errors='ignore').strip())
    #     else:
    #         # 读取超时，认为一次响应结束
    #         break
            
    print ("已发送 config = %d" %config)
    # 打印本次的响应
    # print("--- 本次响应 ---")
    # for response_line in response_lines:
    #     print(response_line)
    # print("------------------")
    
    return Trimming



import pandas as pd
import numpy as np
import serial
import time
from pymeasure.instruments.keithley import Keithley2182

Nanovoltmeter = Keithley2182("GPIB::5")
Nanovoltmeter.reset()
Nanovoltmeter.voltage_nplc = 0.01
Nanovoltmeter.active_channel = 1             # Sets channel 1 for active measurement
Nanovoltmeter.channel_function = 'voltage'   # Configures active channel for voltage measurement

ser = serial.Serial('COM4', 115200, timeout=0.5)

output_filename = 'measurement_data.csv'
column_names = ['Trimming'] + [f'res_volt_{j+1}' for j in range(2)] + ['volt_average']

for i in range (8):
    Trimming = send_config (ser, i)
    res_volts_list = []
    for j in range (2):
        res_volt = round (Nanovoltmeter.voltage, 10)
        res_volts_list.append(res_volt)

    # 计算6次电压的平均值
    volt_avg = np.mean(res_volts_list)
    # 将 Trimming 值, 6个电压读数, 和平均值合并成一个数据行
    row = [Trimming] + res_volts_list + [volt_avg]
    # 将单行数据转换为 DataFrame
    df_row = pd.DataFrame([row], columns=column_names)

    # # 如果是第一次写入 (i == 0)，则正常写入并包含表头 (header)
    # if i == 0:
    #     df_row.to_csv(output_filename, index=False, mode='w') # 'w' for write mode
    # # 如果不是第一次，则使用追加模式 ('a') 写入，并且不包含表头
    # else:
    #     df_row.to_csv(output_filename, index=False, mode='a', header=False)

已发送 config = 0


d:\anaconda3\envs\Lab-Protocol\Lib\site-packages\pymeasure\adapters\visa.py:172: UserWarning: read string doesn't end with termination characters
  return self.connection.read(**kwargs)


已发送 config = 1
已发送 config = 2
已发送 config = 3
已发送 config = 4
已发送 config = 5
已发送 config = 6
已发送 config = 7


In [ ]:
import pandas as pd
import numpy as np
import serial
import time
from pymeasure.instruments.keithley import Keithley2182

Nanovoltmeter = Keithley2182("GPIB::5")
Nanovoltmeter.reset()
Nanovoltmeter.voltage_nplc = 0.01
Nanovoltmeter.active_channel = 1             # Sets channel 1 for active measurement
Nanovoltmeter.channel_function = 'voltage'   # Configures active channel for voltage measurement

ser = serial.Serial('COM4', 115200, timeout=0.5)

In [ ]:
output_filename = 'measurement_data.csv'
column_names = ['Trimming'] + [f'res_volt_{j+1}' for j in range(2)] + ['volt_average']

for i in range (8):
    Trimming = send_config (ser, i)
    res_volts_list = []
    for j in range (2):
        res_volt = round (Nanovoltmeter.voltage, 10)
        res_volts_list.append(res_volt)

    # 计算6次电压的平均值
    volt_avg = np.mean(res_volts_list)
    # 将 Trimming 值, 6个电压读数, 和平均值合并成一个数据行
    row = [Trimming] + res_volts_list + [volt_avg]
    # 将单行数据转换为 DataFrame
    df_row = pd.DataFrame([row], columns=column_names)

    # # 如果是第一次写入 (i == 0)，则正常写入并包含表头 (header)
    # if i == 0:
    #     df_row.to_csv(output_filename, index=False, mode='w') # 'w' for write mode
    # # 如果不是第一次，则使用追加模式 ('a') 写入，并且不包含表头
    # else:
    #     df_row.to_csv(output_filename, index=False, mode='a', header=False)